In [1]:
import polars as pl

In [2]:
df = pl.read_csv(
    'https://gist.githubusercontent.com/ritchie46/'
    'cac6b337ea52281aa23c049250a4ff03/raw/'
    '89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv')
df.head()

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega V...","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


In [6]:
df.select([
    'Type 1',
    'Type 2',
    pl.col('Attack').mean().over('Type 1').alias('mean_attack_by_type'),
    pl
    .col('Defense')
    .mean()
    .over(['Type 1', 'Type 2'])
    .alias('mean_def_by_both_types'),
    pl.col('Attack').mean().alias('mean_attack')])

Type 1,Type 2,mean_attack_by_type,mean_def_by_both_types,mean_attack
str,str,f64,f64,f64
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693
"""Fire""","""Dragon""",88.642857,111.0,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693


In [9]:
filtered = (
    df
    .filter(pl.col('Type 2') == 'Psychic')
    .select(['Name', 'Type 1', 'Speed']))
filtered

Name,Type 1,Speed
str,str,i64
"""Slowpoke""","""Water""",15
"""Slowbro""","""Water""",30
"""SlowbroMega Sl...","""Water""",30
"""Exeggcute""","""Grass""",40
"""Exeggutor""","""Grass""",55
"""Starmie""","""Water""",115
"""Jynx""","""Ice""",95


In [10]:
(filtered
 .with_columns([
     pl.col(['Name', 'Speed']).sort(reverse=True).over('Type 1')]))

Name,Type 1,Speed
str,str,i64
"""Starmie""","""Water""",115
"""Slowpoke""","""Water""",30
"""SlowbroMega Sl...","""Water""",30
"""Exeggutor""","""Grass""",55
"""Exeggcute""","""Grass""",40
"""Slowbro""","""Water""",15
"""Jynx""","""Ice""",95
